<a href="https://colab.research.google.com/github/dstrick97/cse380-notebooks/blob/master/09_2_Ponder_and_Prove_Data_Compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ponder and Prove Data Compression
#### Due: Saturday, 6 March 2021, 11:59 pm.

# TODO Explore Huffman Trees and Huffman Codes


Your task is examine how to compress a *special piece of information* as compactly as possible, and **calculate various compression ratios**.

Recall that the **compression ratio** of a variable-length encoding like Huffman encoding is the percentage $100(f - v)/f$, where $f$ is the number of bits per symbol of the smallest **fixed**-length encoding, and $v$ is the average number of bits per symbol with the variable-length encoding.

For example, if there were 9 different symbols in a message, $f=4$ is the number of bits of the smallest fixed-length encoding, because $2^3 = 8$ (not enough for $9$) and $2^4 = 16$ (enough and to spare). If the variable-length encoding of the message had $v=3.12$, the compression ratio would be $100(4 - 3.12)/4 \approx 22\%$.

Note that calculating the average number of bits per symbol is not strictly necessary. That's because an alternate and equivalent way is to calculate $100(ft - vt)/ft$, where $ft$ is the **total** number of bits encoded with the fixed encoding, and $vt$ is the **total** number of bits encoded with the variable-length encoding.

The *special piece of information* to be compressed is a list of the first ten million primes. This is a list that starts

|    |
|----|
|  2 |
|  3 |
|  5 |
|  7 |
| 11 |
| 13 |
| 17 |
| 19 |
| 23 |
| 29 |

  and ends

|           |
|-----------|
| 179424551 |
| 179424571 |
| 179424577 |
| 179424601 |
| 179424611 |
| 179424617 |
| 179424629 |
| 179424667 |
| 179424671 |
| 179424673 |

As ASCII text stored in a file with one prime per line, the size of this data file is slightly over 89 megabytes. The goal is to compress this down to just over 5 megabytes (5589056 bytes, to be exact). That's a 94% compression ratio!

Standard compression tools can only get about a 73% compression ratio for this ASCII data. A more clever approach is needed. Instead of compressing the list of prime numbers, compress a list of the *gaps* between them!

It doesn't save much, just the unique (occurring only once) gap size of 1 between 2 and 3, but in the spirit of de Polignac's conjecture that every *even* number appears infinitely often as a gap between consecutive primes, just consider the even-sized gaps. The result will be a list that starts with 2 (the difference between 5 and 3), 2 (the difference between 7 and 5), 4 (the difference between 11 and 7), 2 (the difference between 13 and 11), 4 (the difference between 17 and 13), 2 (the difference between 19 and 17), 4 (the difference between 23 and 19), and 6 (the difference between 29 and 23).

Generating this data is the first task. The algorithm for doing so is very straightforward:

1. Find the gaps between consecutive odd primes.
2. Store these gaps as a list of even numbers.

Tabulating the results, the first ten gaps and the last ten gaps are as follows, where the numbers after the equals signs are the gaps to list:

|                 |
|-----------------|
|  5  -   3  =  2 |
|  7  -   5  =  2 |
| 11  -   7  =  4 |
| 13  -  11  =  2 |
| 17  -  13  =  4 |
| 19  -  17  =  2 |
| 23  -  19  =  4 |
| 29  -  23  =  6 |
| 31  -  29  =  2 |
| 37  -  31  =  6 |

|                                |
|--------------------------------|
| 179424551  -  179424533  =  18 |
| 179424571  -  179424551  =  20 |
| 179424577  -  179424571  =   6 |
| 179424601  -  179424577  =  24 |
| 179424611  -  179424601  =  10 |
| 179424617  -  179424611  =   6 |
| 179424629  -  179424617  =  12 |
| 179424667  -  179424629  =  38 |
| 179424671  -  179424667  =   4 |
| 179424673  -  179424671  =   2 |

As a correctness check, see if your generated list of gaps has length 9999998.

The next step is to count how many times each gap size occurs, so that for the Huffman encoding scheme, the larger the frequency of occurrence, the smaller the number of bits encoding that gap size.

As a correctness check, here are the first ten and the last ten gap counts:

|  Gap | Count   |
|------|---------|
|    2 |  738597 |
|    4 |  738717 |
|    6 | 1297540 |
|    8 |  566151 |
|   10 |  729808 |
|   12 |  920661 |
|   14 |  503524 |
|   16 |  371677 |
|   18 |  667734 |
|   20 |  354267 |
|      |         |
|  190 |       1 |
|  192 |       3 |
|  194 |       1 |
|  196 |       1 |
|  198 |       6 |
|  202 |       2 |
|  204 |       3 |
|  210 |       4 |
|  220 |       1 |
|  222 |       1 |

Note two things from these partial gap counts:

1. Small even numbers (< 100) are well represented, larger ones (< 1000) less so.
2. Ten million primes aren't enough to have *every* even number represented; for example, 200, 206, 208, 212, 214, 216, and 218 do not appear even once.


In [3]:
#Thank you Kyle!
!pip install pyprimesieve
from math import log10
from math import ceil
from math import floor
import pyprimesieve
tmp = pyprimesieve.primes_nth(10000000)
primes = pyprimesieve.primes(tmp+1)
gaps = [*map(lambda i:primes[i]-primes[i-1],range(1,10000000))]
gaps.remove(1)

  Created wheel for pyprimesieve: filename=pyprimesieve-0.1.6-cp37-cp37m-linux_x86_64.whl size=368949 sha256=b8f09400199cdcd16902f42fa3fbcc8f3a83b48a041f17307a5f99ca0d4f3994
  Stored in directory: /root/.cache/pip/wheels/c4/63/2b/a485079de882a375d28a4dc141386c76ea9a6aaad505f2198b
Successfully built pyprimesieve


In [4]:
gapOccurrence = {2:0}
for i in range(2, 223):
  gapOccurrence[i] = 0
  i += 1
for x in gaps:
  gapOccurrence[x] += 1

print(2, gapOccurrence[2], 222, gapOccurrence[222])

2 738597 222 1


The number of occurances of each gap size is now stored within the gapOccurrence dictionary

# TODO Determine Exact Size of Data to be Compressed


Without actually doing it, imagine creating an ASCII file containing the first ten million primes, represented in decimal, one prime per line. Calculate the size of this file, so you can show an exceptional compression ratio from it (see below).

Using a binary encoding instead of ASCII, each prime requires 32 bits (4 bytes), so the size of a binary file is easily determined.

Using a fixed-width encoding of the gap counts, however, requires knowing how many different gap sizes there are, after which the calculation is straightforward.

In [5]:
def get_num_digits_no_str(n):
  return floor(log10(n)) + 1

In [6]:
total_size_in_digits = sum(map(lambda p: get_num_digits_no_str(p), primes))

In [7]:
total_size_in_bits = total_size_in_digits * 8
(total_size_in_digits + 10 ** 7) / 2 ** 20

89.15371894836426

# TODO Use Functional Python


You are encouraged to use the [anytree](https://pypi.org/project/anytree) Python library, which has a nice exporter by way of which you can graphically view trees. (You may recall using this in DM1, and thus know that **anytree** depends on [graphviz](https://graphviz.org), which you also used.)

This library uses the object-oriented features of Python to create and visualize trees. You are encouraged to use the functional features of Python as much as possible, achieving your results not by using some existing third-party libraries for building Huffman Trees and Codes, but writing your own code as cleanly and elegantly as you can.

In [14]:
!pip install anytree
from anytree import Node, RenderTree, PreOrderIter, Walker
from anytree.util import leftsibling, rightsibling
from anytree.exporter.dotexporter import DotExporter
from collections import Counter
from queue import PriorityQueue
from math import ceil, floor, log, log2
import pyprimesieve

class GapNode(Node):
  def __lt__(self, other):
    return self.count < other.count

node_counter = 0
def next_node_name():
  global node_counter
  name = 'gn' + str(node_counter)
  node_counter += 1
  return name

def new_node(gp, ct):
  return GapNode(next_node_name(), gap = gp, count = ct)

def new_internal_node(left, right):
  return GapNode(next_node_name(), children = [left, right],
                 gap = 0, count = left.count + right.count)

def make_huffman_tree(gaps_list):
  gap_dict = Counter(gaps_list)
  q = PriorityQueue()
  for key, val in gap_dict.items():
    q.put(new_node(key, val))

  while q.qsize() > 1:
    left = q.get()
    right = q.get()
    q.put(new_internal_node(left, right))

  return q.get()

def get_codes(root):
  leaves = [node for node in PreOrderIter(root, filter_=lambda n: not n.children)]
  codes = {}
  w = Walker()
  for leaf in leaves:
    path = w.walk(leaf, root)[0]
    code = ''
    for node in path:
      code = ('1' if leftsibling(node) else '0') + code
    codes[leaf.gap] = tuple([code, leaf.count])
  return codes

def compression_ratio(f, v):
  return 100 * (f - v) / f

def get_encoded_size(codes):
  return sum([len(code) * count for gap, (code, count) in codes.items()])

def get_fixed_size(codes):
  num_keys = len(codes)
  num_bits_per_key = ceil(log(num_keys, 2))
  return sum([num_bits_per_key * count for gap, (code, count) in codes.items()])

def report(codes):
  return compression_ratio(get_fixed_size(codes), get_encoded_size(codes))

def test_up_to(primes, upper, results):
  list_of_gaps = get_list_of_gaps(primes[:upper])
  print(primes[:upper])
  print(list_of_gaps)
  root = make_huffman_tree(list_of_gaps)
  print(RenderTree(root))
  DotExporter(root).to_picture(f'gap-tree-{upper:02d}.png')
  codes = get_codes(root)
  print(upper, '-->', get_encoded_size(codes))
  cr = round(report(codes))
  results[upper - 1] = cr # adjust since upper is the number of primes, -1 to make it just odd primes

#thank you Claire for helping with these function
def get_bin_size(codes):
  return 32 * len(primes)

#thank you Bretton for helping create this function
def get_ascii_size(primes):
  return sum([len(str(prime) + '\n') * 8 for prime in primes])



In [9]:
treeroot = make_huffman_tree(gaps)
codes = get_codes(treeroot)
print("Fixed size:", get_fixed_size(codes))
print("Encoded size:", get_encoded_size(codes))
print("From Fixed compression ratio:",report(codes))

Fixed size: 69999986
Encoded size: 44712373
From Fixed compression ratio: 36.12516865360516


In [10]:
print("Binary size:",get_bin_size(codes))
b = compression_ratio(get_bin_size(codes), get_encoded_size(codes))
print("From Binary compression ratio:", b )

Binary size: 320000000
From Binary compression ratio: 86.0273834375


In [11]:
print("ASCII string size:", get_ascii_size(primes))
a = compression_ratio(get_ascii_size(primes), get_encoded_size(codes))
print("From ASCII compression ratio:", a)

ASCII string size: 747875600
From ASCII compression ratio: 94.02141572742846


# TODO Achieve Target Compression Ratios

Your solution should correctly compute the following three compression ratios:

| Ratio       | Value              |
|-------------|--------------------|
| From fixed  | 36.125168653605158 |
| From binary |              86.03 |
| From ASCII  |              94.02 | 


See above for compression ratios

# TODO My Report on What I Did and What I Learned

## Fun


I had fun learning more about Huffman encoding. Parts of it were hard to understand but it was fun to see how pieces of code linked together to create an encoded tree. Working in a group was fun as well as we worked together to solve the problems.

## New

I learned a lot more about Huffman trees. I remember learning a little about them in DM1, but most of my understanding of them comes from Data Structures where we examined them in relation to nodes and trees. I learned much more about why they work and where they are useful instead of just knowing that they exist.

## Meaningful


I learned about why encoding is important. Seeing a 94% compression of a file is a good example of why I should care. Being able to compress and expand data to fit needs and situations is a powerful tool and it is one I might need in the future. Knowing that it is possible and beginning to understand how it works feels like a very important first step.

## Other

This week I got to work with Hannah Parker, Bretton Steiner, and Claire Hocker. They were great in helping me understand how the compression worked and what variables to use in which cases.

# TODO What is True?
Click on each warranted checkbox to toggle it to True (or back to False). 

NOTE: *This only works in Colab. If you run it in some other Jupyter notebook client/server environment you may have to change False to True (or vice versa) manually.*

This self-assessment is subject to revision by a grader.

In [ ]:
#@markdown ## What is True about what I did?
#@markdown ### I had fun.
cb00 = True #@param {type:'boolean'}
#@markdown ### I learned something new.
cb01 = True #@param {type:'boolean'}
#@markdown ### I achieved something meaningful, or something I can build upon at a later time.
cb02 = True #@param {type:'boolean'}
#@markdown ## What is True about my report?
#@markdown ### I wrote a sufficient number of well-written sentences.
cb03 = True #@param {type:'boolean'}
#@markdown ### My report is free of mechanical infelicities.
cb04 = True #@param {type:'boolean'}
#@markdown ### I used Grammarly (or something better described in my report) to check for MIs.
cb05 = True #@param {type:'boolean'}
#@markdown ### I reported on any connections I found between these problems and something I already know. 
cb06 = True #@param {type:'boolean'}
#@markdown ### I reported who were and what contribution each of my collaborators made.
cb07 = True #@param {type:'boolean'}
#@markdown ## What is True about my calculations?
#@markdown ### I correctly calculated the number of times each gap size occurs. 
cb08 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the number of bits per gap size with a fixed encoding.
cb09 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the total number of bits encoded with the Huffman encoding.
cb10 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the total number of bits encoded with the fixed encoding.
cb11 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the compression ratio from this fixed encoding.
cb12 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the size of the first ten million primes encoded as 32-bit integer binary data.
cb13 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the compression ratio from the binary size.
cb14 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the size of the first ten million primes encoded as ASCII data.
cb15 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the compression ratio from the ASCII size (just the primes, nothing else).
cb16 = True #@param {type:'boolean'}